In [1]:
using PyCall
unshift!(PyVector(pyimport("sys")["path"]), "");
@pyimport GLS_Apr_weekday_AM

INFO: Recompiling stale cache file /home/jzh/.julia/lib/v0.4/PyCall.ji for module PyCall.


No dicts found; please check load_dicts...


In [2]:
xi_list = GLS_Apr_weekday_AM.xi_list
P = GLS_Apr_weekday_AM.P
L = GLS_Apr_weekday_AM.L  # dimension of xi

314

In [3]:
size(P, 1), size(P, 2)

(56,314)

In [5]:
using JuMP
using Ipopt

mGLSJulia = Model(solver=IpoptSolver())

@variable(mGLSJulia, lam[1:size(P,1)] >= 0)

@variable(mGLSJulia, p[1:size(P,1), 1:size(P,2)] >= 0)

for i = 1:size(P,1)
    for j = 1:size(P,2)
        if P[i,j] == 0
            @constraint(mGLSJulia, p[i,j] == 0)
        end
    end
end
            
for i = 1:size(P,1)
    @NLconstraint(mGLSJulia, sum(p[i,j] for j = 1:size(P,2)) == 1)
end

for l = 1:L
    @NLconstraint(mGLSJulia, sum(p[i,l] * lam[i] for i = 1:size(P,1)) == xi_list[l])
end
    
@NLobjective(mGLSJulia, Min, sum(p[1,j] for j = 1:size(P,2)))  # play no actual role, but could not use zero objective

solve(mGLSJulia)

INFO: Precompiling module JuMP...
    push!(DataStructures.Accumulator{#T<:Any, #V<:Number}, #T<:Any) at /home/jzh/.julia/v0.4/DataStructures/src/accumulator.jl:54
is ambiguous with: 
    push!(Base.Associative, Base.Pair) at dict.jl:294.
To fix, define 
    push!(DataStructures.Accumulator{K<:Any, #V<:Number}, _<:Base.Pair)
before the new definition.
INFO: Recompiling stale cache file /home/jzh/.julia/lib/v0.4/JuMP.ji for module JuMP.
INFO: Precompiling module Ipopt...


LoadError: syntax: missing comma or ) in argument list

In [ ]:
getvalue(lam)

In [ ]:
getObjectiveValue(mGLSJulia)

In [ ]:
GLS_Apr_weekday_AM.saveDemandVec(getvalue(lam))